Install the tweepy library

In [1]:
# !pip install tweepy
# !pip install mysqlclient

Import required libraries

In [2]:
import os
import re
import time

import tweepy as tw
import pandas as pd
from datetime import datetime, timedelta, timezone
import MySQLdb


# For sentiment analysis of tweets
from textblob import TextBlob

Create Tweeter Consumer Keys and Access Tokens 

In [3]:
consumer_key= 'NFIO7bIgwQpdCZ7'
consumer_secret= '2QegNrIpcGCW6nCWFTL5JEsrDCmYZp'
access_token= '3049227557-36ln3qak9xbxHvgNos45cf3u'
access_token_secret= 'pZ4Y0p31graKrAYAy41mYztvI'

Initialize AuthHandler

In [4]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

Define search words and date

In [5]:
search_words = "#Covid"
date_since = "2018-11-16"

In [6]:
tweets = tw.Cursor(api.search, q=search_words, lang="en", since=date_since).items(5)

In [7]:
tweets

Iterate through tweets using for loop

In [8]:
from datetime import datetime, timedelta
from email.utils import parsedate_tz

s='Mon Dec 07 17:03:03 +0000 2020'
def to_datetime(datestring):
    time_tuple = parsedate_tz(datestring.strip())
    dt = datetime(*time_tuple[:6])
    return dt - timedelta(seconds=time_tuple[-1])

print(to_datetime(s))
print(to_datetime(s).time().strftime("%H:%M:%S"))

x=datetime.strptime('12/31/13 00:12', "%m/%j/%y %H:%M").time().strftime("%H:%M:%S")

print(x)

2020-12-07 17:03:03
17:03:03
00:12:00


Retrieve tweets as a collection of list

In [9]:
[tweet.text for tweet in tweets]

['#UK dreams for getting rid of #covid-19! https://t.co/NaHURX2nVc',
 'RT @VABVOX: If you were looking for the worst people in Congress, they are:\n▪Marsha Blackburn \n▪Ted Cruz\n▪Ron Johnson\n▪Mike Lee\n▪Rand Paul…',
 'RT @NWT_CPHO: THREAD 1/2 – The #CovidAlert App is LIVE in #NWT. By exchanging anonymous Bluetooth codes with nearby phones, the app can tel…',
 "Latest news on #RohitSharma\n\n #Sydney #Covid cluster unlikely to affect #Rohit's participation \n\n@CricCrazyJohns… https://t.co/rXvTYbNmdm",
 'Look at this idiot, trying to take credit for #Covid vaccine that be initially called a hoax https://t.co/kisQR0ry7A']

Search Tweets without Retweets

In [10]:
new_search = search_words + " -filter:retweets"

In [11]:
new_search

'#Covid -filter:retweets'

In [12]:
tweets = tw.Cursor(api.search, q=new_search, lang="en",since=date_since).items(5)

In [13]:
[tweet.text for tweet in tweets]

["Latest news on #RohitSharma\n\n #Sydney #Covid cluster unlikely to affect #Rohit's participation \n\n@CricCrazyJohns… https://t.co/rXvTYbNmdm",
 'Look at this idiot, trying to take credit for #Covid vaccine that be initially called a hoax https://t.co/kisQR0ry7A',
 'Why are we shipping money to Pakistan when Americans are starving because of the #COVID shutdowns? #stimulus',
 '@WhiteHouse We should have gotten a #COVID19 vaccine sooner but \n@realDonaldTrump downplayed the dangers of #COVID… https://t.co/7nVYadq0kq',
 'Markets fall as more nations join #UK travel ban over #Covid variant; lorry drivers stranded, supermarkets warn of… https://t.co/CjmEs9hRoq']

Get username and location

In [14]:
tweets = tw.Cursor(api.search,q=new_search, lang="en", since=date_since).items(5)

In [15]:
users_locs = [[tweet.user.screen_name,tweet.user.location,tweet.text] for tweet in tweets]

In [16]:
users_locs

[['Being_SAi143',
  'мuмвɑᎥ',
  "Latest news on #RohitSharma\n\n #Sydney #Covid cluster unlikely to affect #Rohit's participation \n\n@CricCrazyJohns… https://t.co/rXvTYbNmdm"],
 ['dangainor',
  'Reston, VA.',
  'Why are we shipping money to Pakistan when Americans are starving because of the #COVID shutdowns? #stimulus'],
 ['soloyochapin',
  'Chicago, IL',
  '@WhiteHouse We should have gotten a #COVID19 vaccine sooner but \n@realDonaldTrump downplayed the dangers of #COVID… https://t.co/7nVYadq0kq'],
 ['Diplomacy140',
  'London, England',
  'Markets fall as more nations join #UK travel ban over #Covid variant; lorry drivers stranded, supermarkets warn of… https://t.co/CjmEs9hRoq'],
 ['The_Truth_II',
  'Praia da Luz,  Portugal ',
  'How long are we prepared to live like this? #covid #lockdown #tiersystem #masks #housearrest #roadblocks… https://t.co/phIYFc9jvw']]

Create a Pandas Dataframe From A List of Tweet Data

In [17]:
tweet_df = pd.DataFrame(data=users_locs, columns=['user', "location","tweet"])

In [18]:
tweet_df

,user,location,tweet
0,Being_SAi143,мuмвɑᎥ,Latest news on #RohitSharma\n\n #Sydney #Covid...
1,dangainor,"Reston, VA.",Why are we shipping money to Pakistan when Ame...
2,soloyochapin,"Chicago, IL",@WhiteHouse We should have gotten a #COVID19 v...
3,Diplomacy140,"London, England",Markets fall as more nations join #UK travel b...
4,The_Truth_II,"Praia da Luz, Portugal",How long are we prepared to live like this? #c...


## Working With The Twitter Streaming API

With Streaming data we need to do the following:
1. Create database connection
2. Create a persistent connection to the Twitter API, and read each connection incrementally.
3. Process tweets quickly, store them in the database, and don’t let your program get backed up.
4. Handle errors and other issues properly.

Connect to MySql Database

In [19]:
#                           (server,MySQL username,MySQL pass, Database name).
connection = MySQLdb.connect("localhost","root","root","social_analytic")
cur = connection.cursor()